In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

/tmp/ipykernel_4489/2959942666.py:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  train=pd.read_csv('train.csv')


In [48]:
print(train.shape,test.shape)

(177024, 508) (3644, 507)


In [3]:
train.columns

Index(['id', 'p_num', 'time', 'bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40',
       'bg-5:35', 'bg-5:30', 'bg-5:25',
       ...
       'activity-0:40', 'activity-0:35', 'activity-0:30', 'activity-0:25',
       'activity-0:20', 'activity-0:15', 'activity-0:10', 'activity-0:05',
       'activity-0:00', 'bg+1:00'],
      dtype='object', length=508)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177024 entries, 0 to 177023
Columns: 508 entries, id to bg+1:00
dtypes: float64(433), object(75)
memory usage: 686.1+ MB


In [5]:
train.isnull().mean()*100

id                0.000000
p_num             0.000000
time              0.000000
bg-5:55          15.395653
bg-5:50          10.445476
                   ...    
activity-0:15    98.457271
activity-0:10    98.453882
activity-0:05    98.444844
activity-0:00    98.453882
bg+1:00           0.000000
Length: 508, dtype: float64

In [6]:
(train.isnull().mean()*100).values.mean()

44.80774393863051

In [7]:
(test.isnull().mean()*100).values.mean()

46.67958136040548

In [8]:
# len([col for col in train_cleaned.columns if col.startswith('carbs')])
train[[col for col in train.columns if col.startswith('bg')]].isnull().mean()*100

bg-5:55    15.395653
bg-5:50    10.445476
bg-5:45     7.716468
bg-5:40    15.397912
bg-5:35    10.603082
             ...    
bg-0:15     1.848337
bg-0:10    14.586723
bg-0:05    13.804908
bg-0:00     1.522957
bg+1:00     0.000000
Length: 73, dtype: float64

## 降低記憶體消耗

In [9]:
def reduce_memory(df):
    ##bytes to MB
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage before optimization: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            if str(col_type).startswith("int"):
                df[col] = pd.to_numeric(df[col], downcast="integer")
            elif str(col_type).startswith("float"):
                df[col] = pd.to_numeric(df[col], downcast="float")
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization: {end_mem:.2f} MB")
    print(f"Reduced by {(100 * (start_mem - end_mem) / start_mem):.1f}%")

    return df

# Apply memory reduction
train = reduce_memory(train)
test = reduce_memory(test)

Memory usage before optimization: 686.10 MB
Memory usage after optimization: 311.18 MB
Reduced by 54.6%
Memory usage before optimization: 14.10 MB
Memory usage after optimization: 6.46 MB
Reduced by 54.2%


## 特徵篩選

In [46]:
target_column = 'bg+1:00'
numerical_columns = train.select_dtypes(include=['number']).columns
if target_column not in numerical_columns:
    raise ValueError(f"Target column '{target_column}' is not numerical.")
correlation_matrix = train[numerical_columns].corr(method='spearman')
correlations_with_target = correlation_matrix[target_column]
print(correlations_with_target)

bg-5:55      0.180896
bg-5:50      0.167134
bg-5:45      0.169330
bg-5:40      0.181056
bg-5:35      0.167420
               ...   
cals-0:15   -0.045281
cals-0:10   -0.046652
cals-0:05   -0.047248
cals-0:00   -0.047158
bg+1:00      1.000000
Name: bg+1:00, Length: 433, dtype: float64


In [47]:
for threshold in np.arange(0.1, 1.0, 0.1):
    # threshold=0.2
    threshold=round(threshold,1)
    high_correlation_features = correlations_with_target[correlations_with_target.abs() > threshold].index.tolist()
    high_correlation_features.remove(target_column)
    print("所有特徵有{}個，相關性大於{}的特徵有{}個：".format(len(train_cleaned.columns)-1,threshold,len(high_correlation_features)))

所有特徵有293個，相關性大於0.1的特徵有128個：
所有特徵有293個，相關性大於0.2的特徵有40個：
所有特徵有293個，相關性大於0.3的特徵有20個：
所有特徵有293個，相關性大於0.4的特徵有12個：
所有特徵有293個，相關性大於0.5的特徵有6個：
所有特徵有293個，相關性大於0.6的特徵有2個：
所有特徵有293個，相關性大於0.7的特徵有0個：
所有特徵有293個，相關性大於0.8的特徵有0個：
所有特徵有293個，相關性大於0.9的特徵有0個：


## 刪除空值過多欄位

In [12]:
train_miss=train.isnull().mean()*100
miss_thresh=49
train_dropcol=train_miss[train_miss>miss_thresh].index

In [13]:
print(len(train_dropcol))

216


In [14]:
train_cleaned = train.drop(columns=train_dropcol)
test_cleaned = test.drop(columns=train_dropcol)

In [15]:
print(train_cleaned.shape)
print(test_cleaned.shape)

(177024, 292)
(3644, 291)


## 刪除不必要欄位id

In [16]:
train_cleaned = train_cleaned .drop('id',axis=1)
test_cleaned = test_cleaned.drop('id',axis=1)

In [17]:
print(train_cleaned.shape)
print(test_cleaned.shape)

(177024, 291)
(3644, 290)


## 日期特徵擷取

In [18]:
def extractime(df):
    df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time
    hour = df['time'].apply(lambda x: x.hour)
    minute = df['time'].apply(lambda x: x.minute)
    new_columns = {
        'hour_sin': np.sin(2 * np.pi * hour / 24),
        'hour_cos': np.cos(2 * np.pi * hour / 24),
        'minute_sin': np.sin(2 * np.pi * minute / 60),
        'minute_cos': np.cos(2 * np.pi * minute / 60),
    }
    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)
    df.drop(['time'], axis=1, inplace=True)
    return df

In [19]:
train_cleaned=extractime(train_cleaned)
test_cleaned=extractime(test_cleaned)

In [20]:
print(train_cleaned.shape)
print(test_cleaned.shape)

(177024, 294)
(3644, 293)


## 患者Id處理

In [21]:
train_cleaned['p_num'].unique()

['p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p10', 'p11', 'p12']
Categories (9, object): ['p01', 'p02', 'p03', 'p04', ..., 'p06', 'p10', 'p11', 'p12']

In [22]:
test_cleaned['p_num'].unique()

['p01', 'p02', 'p04', 'p05', 'p06', ..., 'p18', 'p19', 'p21', 'p22', 'p24']
Length: 15
Categories (15, object): ['p01', 'p02', 'p04', 'p05', ..., 'p19', 'p21', 'p22', 'p24']

In [23]:
set(train_cleaned['p_num'].unique()).issubset(set(test_cleaned['p_num'].unique()))

False

In [24]:
unique_pnums = list(set(train_cleaned['p_num'].unique()).union(set(test_cleaned['p_num'].unique())))
pnum_mapping = {value: idx for idx, value in enumerate(unique_pnums)}
test_cleaned['p_num'] = test_cleaned['p_num'].map(pnum_mapping)

In [25]:
unique_pnums

['p10',
 'p22',
 'p01',
 'p06',
 'p19',
 'p24',
 'p04',
 'p18',
 'p02',
 'p03',
 'p15',
 'p21',
 'p05',
 'p11',
 'p12',
 'p16']

In [26]:
pnum_mapping

{'p10': 0,
 'p22': 1,
 'p01': 2,
 'p06': 3,
 'p19': 4,
 'p24': 5,
 'p04': 6,
 'p18': 7,
 'p02': 8,
 'p03': 9,
 'p15': 10,
 'p21': 11,
 'p05': 12,
 'p11': 13,
 'p12': 14,
 'p16': 15}

In [27]:
train_cleaned['p_num'] = train_cleaned['p_num'].map(pnum_mapping)

## 類別欄位(activity)檢查

In [28]:
act_col=[col for col in train_cleaned.columns if col.startswith('activity')]

In [29]:
print(len(act_col))

0


## 補空值

In [30]:
train_cleaned.isnull().mean()*100

p_num          0.000000
bg-5:55       15.395653
bg-5:50       10.445476
bg-5:45        7.716468
bg-5:40       15.397912
                ...    
bg+1:00        0.000000
hour_sin       0.000000
hour_cos       0.000000
minute_sin     0.000000
minute_cos     0.000000
Length: 294, dtype: float64

In [31]:
test_cleaned.isnull().mean()*100

p_num          0.000000
bg-5:55       27.003293
bg-5:50       15.998902
bg-5:45       13.336992
bg-5:40       27.058178
                ...    
cals-0:00     14.324918
hour_sin       0.000000
hour_cos       0.000000
minute_sin     0.000000
minute_cos     0.000000
Length: 293, dtype: float64

In [32]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
train_imputed = imputer.fit_transform(train_cleaned.drop('bg+1:00',axis=1))
test_imputed = imputer.transform(test_cleaned)

In [33]:
train_target=train_cleaned['bg+1:00']

In [34]:
columns = list(train_cleaned.columns)  
columns.remove('bg+1:00')
train_cleaned = pd.DataFrame(train_imputed, columns=columns)
test_cleaned = pd.DataFrame(test_imputed, columns=test_cleaned.columns)

In [35]:
train_cleaned['bg+1:00']=train_target

In [36]:
print(train_cleaned.shape)
print(test_cleaned.shape)

(177024, 294)
(3644, 293)


In [37]:
train_miss_summary = train_cleaned.isnull().sum()
if train_miss_summary.sum() == 0:
    print("所有欄位皆沒有空值")
else:
    print("以下欄位有空值：")
    print(train_miss_summary[train_miss_summary > 0])

所有欄位皆沒有空值


In [38]:
test_miss_summary = test_cleaned.isnull().sum()
if test_miss_summary.sum() == 0:
    print("所有欄位皆沒有空值")
else:
    print("以下欄位有空值：")
    print(test_miss_summary[test_miss_summary > 0])

所有欄位皆沒有空值


In [39]:
# train_cleaned.to_csv('train_cleaned.csv',index=False)
# test_cleaned.to_csv('test_cleaned.csv',index=False)

## 分割資料

In [40]:
X= train_cleaned.drop('bg+1:00', axis=1)
y = train_cleaned['bg+1:00']

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2 , random_state = 42)

## 模型測試

In [42]:
from xgboost import XGBRegressor
xgboostModel = XGBRegressor()
xgboostModel.fit(X_train, y_train)
predicted = xgboostModel.predict(X_train)
print("訓練集 Score: ", xgboostModel.score(X_train,y_train))
print("測試集 Score: ", xgboostModel.score(X_val,y_val))

訓練集 Score:  0.7286968485099772
測試集 Score:  0.634854006793717


In [43]:
from sklearn import metrics
train_pred = xgboostModel.predict(X_train)
mse = metrics.mean_squared_error(y_train, train_pred)
print('訓練集 MSE: ', mse)

val_pred = xgboostModel.predict(X_val)
mse = metrics.mean_squared_error(y_val, val_pred)
print('測試集 MSE: ', mse)

訓練集 MSE:  2.4313378
測試集 MSE:  3.302721
